In [1]:
import numpy as np 
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 
print('ready')

ready


In [2]:
df= pd.read_csv('can_geo.csv')
df.sample(5)

Unnamed: 0 Postcode           Borough  \
87          87      M7Y      East Toronto   
40          40      M4J         East York   
44          44      M4N   Central Toronto   
26          26      M3B        North York   
53          53      M5A  Downtown Toronto   

                                        Neighbourhood   Latitude  Longitude  
87  Business Reply Mail Processing Centre 969 Eastern  43.662744 -79.321558  
40                                       East Toronto  43.685347 -79.338106  
44                                      Lawrence Park  43.728020 -79.388790  
26                                    Don Mills North  43.745906 -79.352188  
53                                       Harbourfront  43.654260 -79.360636

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df

Postcode      Borough                                      Neighbourhood  \
0        M1B  Scarborough                                      Rouge,Malvern   
1        M1C  Scarborough               Highland Creek,Rouge Hill,Port Union   
2        M1E  Scarborough                    Guildwood,Morningside,West Hill   
3        M1G  Scarborough                                             Woburn   
4        M1H  Scarborough                                          Cedarbrae   
..       ...          ...                                                ...   
98       M9N         York                                             Weston   
99       M9P    Etobicoke                                          Westmount   
100      M9R    Etobicoke  Kingsview Village,Martin Grove Gardens,Richvie...   
101      M9V    Etobicoke  Albion Gardens,Beaumond Heights,Humbergate,Jam...   
102      M9W    Etobicoke                                          Northwest   

      Latitude  Longitude  
0    43.806686 -79.194353  
1    43.784535 -79.160497  
2    43.763573 -79.188711  
3    43.770992 -79.216917  
4    43.773136 -79.239476  
..         ...        ...  
98   43.706876 -79.518188  
99   43.696319 -79.532242  
100  43.688905 -79.554724  
101  43.739416 -79.588437  
102  43.706748 -79.594054  

[103 rows x 5 columns]

In [4]:
print('The dataframe has {} Boroughs and {} Neighbourhoods.'.format(len(df.Borough.unique()), len(df.Neighbourhood.unique())))

The dataframe has 11 Boroughs and 102 Neighbourhoods.


#### Using the geopy Library for Lat and Lon of Toronto

In [5]:
add= 'Toronto'
geolocator= Nominatim(user_agent="my-app")
location= geolocator.geocode(add)
Latitude= location.latitude
Longitude= location.longitude
print('The geographical coordinates of Toronto are {}, {}'.format(Latitude, Longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207


In [6]:
#Latitude= '43.6532'
#Longitude= '-79.3832'

map_t= folium.Map(location= [Latitude, Longitude], zoom_start=10)

for lat, lon, b, n in zip(df['Latitude'], 
                          df['Longitude'],
                          df['Borough'],
                          df['Neighbourhood']):
    label= '{}, {}'.format(n, b)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= 5,
        popup= label,
        
        color= 'blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_t)

map_t

### Filtering only those Boroughs with the word Toronto in them

In [7]:
boroughs= list(df.Borough.unique())
toronto_boroughs= []

for x in boroughs:
    if "toronto" in x.lower():
        toronto_boroughs.append(x)

toronto_boroughs

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

In [8]:
df_t= df[df['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
df_t.head()

Postcode          Borough                  Neighbourhood   Latitude  \
0      M4E     East Toronto                    The Beaches  43.676357   
1      M4K     East Toronto    The Danforth West,Riverdale  43.679557   
2      M4L     East Toronto  The Beaches West,India Bazaar  43.668999   
3      M4M     East Toronto                Studio District  43.659526   
4      M4N  Central Toronto                  Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

In [9]:
map_1= folium.Map(location=[Latitude,Longitude], zoom_start=10)
for l1, l2, b, n in zip(
                         df_t['Latitude'],
                         df_t['Longitude'],
                         df_t['Borough'],
                         df_t['Neighbourhood']):
    label= '{}, {}'.format(n, b)
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [l1,l2],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_1)
    
map_1

In [29]:
CLIENT_ID= 'your credentials'
CLIENT_SECRET= 'your credentials'
VERSION= 'your credentials'
LIMIT= 100
ACCESS_TOKEN= 'your credentials'
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your credentials
CLIENT_SECRET:your credentials


#### Using the Foursquare API for getting the top 100 venues in a 500 m radius

In [11]:
venues=[]

for l1, l2, post, b, n in zip(df_t['Latitude'],
                              df_t['Longitude'],
                              df_t['Postcode'],
                              df_t['Borough'],
                              df_t['Neighbourhood']):
    url= "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        l1,
        l2,
        radius, 
        LIMIT)
    results= requests.get(url).json()['response']['groups'][0]['items']
    
    for ven in results:
        venues.append((post, 
                        b,
                        n,
                        l1,
                        l2,
                        ven['venue']['name'],
                        ven['venue']['location']['lat'], 
                        ven['venue']['location']['lng'],
                        ven['venue']['categories'][0]['name'] ))
        

In [12]:
venue_df= pd.DataFrame(venues)

venue_df.columns=['Postcode','Borough','Neighbourhood','BoroughLatitutde', 'BoroughLongitude','VenueName', 'VenueLat', 'VenueLon','Venue Category']
venue_df.head()

Postcode       Borough Neighbourhood  BoroughLatitutde  BoroughLongitude  \
0      M4E  East Toronto   The Beaches         43.676357        -79.293031   
1      M4E  East Toronto   The Beaches         43.676357        -79.293031   
2      M4E  East Toronto   The Beaches         43.676357        -79.293031   
3      M4E  East Toronto   The Beaches         43.676357        -79.293031   
4      M4E  East Toronto   The Beaches         43.676357        -79.293031   

                            VenueName   VenueLat   VenueLon     Venue Category  
0                   Glen Manor Ravine  43.676821 -79.293942              Trail  
1  The Big Carrot Natural Food Market  43.678879 -79.297734  Health Food Store  
2                 Grover Pub and Grub  43.679181 -79.297215                Pub  
3                       Upper Beaches  43.680563 -79.292869       Neighborhood  
4                          Dip 'n Sip  43.678897 -79.297745        Coffee Shop

In [13]:
venue_df.groupby(['Postcode', 'Borough', 'Neighbourhood']).count()

BoroughLatitutde  \
Postcode Borough          Neighbourhood                                                          
M4E      East Toronto     The Beaches                                                        5   
M4K      East Toronto     The Danforth West,Riverdale                                       41   
M4L      East Toronto     The Beaches West,India Bazaar                                     19   
M4M      East Toronto     Studio District                                                   41   
M4N      Central Toronto  Lawrence Park                                                      3   
M4P      Central Toronto  Davisville North                                                   9   
M4R      Central Toronto  North Toronto West                                                20   
M4S      Central Toronto  Davisville                                                        33   
M4T      Central Toronto  Moore Park,Summerhill East                                         3   
M4V      Central Toronto  Deer Park,Forest Hill SE,Rathnelly,South Hill,S...                14   
M4W      Downtown Toronto Rosedale                                                           4   
M4X      Downtown Toronto Cabbagetown,St. James Town                                        42   
M4Y      Downtown Toronto Church and Wellesley                                              86   
M5A      Downtown Toronto Harbourfront                                                      47   
M5B      Downtown Toronto Ryerson,Garden District                                          100   
M5C      Downtown Toronto St. James Town                                                   100   
M5E      Downtown Toronto Berczy Park                                                       55   
M5G      Downtown Toronto Central Bay Street                                                79   
M5H      Downtown Toronto Adelaide,King,Richmond                                           100   
M5J      Downtown Toronto Harbourfront East,Toronto Islands,Union Station                  100   
M5K      Downtown Toronto Design Exchange,Toronto Dominion Centre                          100   
M5L      Downtown Toronto Commerce Court,Victoria Hotel                                    100   
M5N      Central Toronto  Roselawn                                                           2   
M5P      Central Toronto  Forest Hill North,Forest Hill West                                 4   
M5R      Central Toronto  The Annex,North Midtown,Yorkville                                 23   
M5S      Downtown Toronto Harbord,University of Toronto                                     35   
M5T      Downtown Toronto Chinatown,Grange Park,Kensington Market                           88   
M5V      Downtown Toronto CN Tower,Bathurst Quay,Island airport,Harbourfr...                17   
M5W      Downtown Toronto Stn A PO Boxes 25 The Esplanade                                   94   
M5X      Downtown Toronto First Canadian Place,Underground city                            100   
M6G      Downtown Toronto Christie                                                          18   
M6H      West Toronto     Dovercourt Village,Dufferin                                       15   
M6J      West Toronto     Little Portugal,Trinity                                           55   
M6K      West Toronto     Brockton,Exhibition Place,Parkdale Village                        22   
M6P      West Toronto     High Park,The Junction South                                      23   
M6R      West Toronto     Parkdale,Roncesvalles                                             13   
M6S      West Toronto     Runnymede,Swansea                                                 38   
M7A      Downtown Toronto Queen's Park                                                      39   
M7Y      East Toronto     Business Reply Mail Processing Centre 969 Eastern                 15   

                                                           

In [14]:
print('There are {} unique categories.'.format(len(venue_df['Venue Category'].unique())))

There are 232 unique categories.


In [15]:
venue_df['Venue Category'].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Coffee Shop',
       'Greek Restaurant', 'Cosmetics Shop', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Pizza Place', 'Bookstore',
       'Restaurant', 'Dessert Shop', 'Juice Bar', 'Bubble Tea Shop',
       'Spa', 'Diner', 'Grocery Store', 'Furniture / Home Store', 'Café',
       'Bakery', 'Caribbean Restaurant', 'American Restaurant',
       'Frozen Yogurt Shop', 'Lounge', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Park', 'Pet Store', 'Steakhouse', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Food & Drink Shop',
       'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Middle Eastern Restaurant', 'Comfort Food Restaurant',
       'Thai Restaurant', 'Seafood Restaurant', 'Stationery Store',
       'Coworking Space', 'Wine Bar'], dtype=object)

In [16]:
toronto_onehot= pd.get_dummies(venue_df[['Venue Category']], prefix="", prefix_sep="")

#print('_____')
#print(toronto_onehot.head())
c1= venue_df['Postcode']
toronto_onehot.insert(0, column= 'Postcode', value=c1)
c2=  venue_df['Borough']
toronto_onehot.insert(1, column='Borough', value=c2)
c3= venue_df['Neighbourhood']
toronto_onehot.insert(2, column='Neighbourhood', value=c3)
#print(toronto_onehot.head())

fixed_col= list(toronto_onehot.columns[:])
toronto_onehot= toronto_onehot[fixed_col]
#print(fixed_col)
#print(toronto_onehot.columns)

toronto_onehot

Postcode       Borough  \
0         M4E  East Toronto   
1         M4E  East Toronto   
2         M4E  East Toronto   
3         M4E  East Toronto   
4         M4E  East Toronto   
...       ...           ...   
1697      M7Y  East Toronto   
1698      M7Y  East Toronto   
1699      M7Y  East Toronto   
1700      M7Y  East Toronto   
1701      M7Y  East Toronto   

                                          Neighbourhood  Afghan Restaurant  \
0                                           The Beaches                  0   
1                                           The Beaches                  0   
2                                           The Beaches                  0   
3                                           The Beaches                  0   
4                                           The Beaches                  0   
...                                                 ...                ...   
1697  Business Reply Mail Processing Centre 969 Eastern                  0   
1698  Business Reply Mail Processing Centre 969 Eastern                  0   
1699  Business Reply Mail Processing Centre 969 Eastern                  0   
1700  Business Reply Mail Processing Centre 969 Eastern                  0   
1701  Business Reply Mail Processing Centre 969 Eastern                  0   

      Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0           0                   0             0               0   
1           0                   0             0               0   
2           0                   0             0               0   
3           0                   0             0               0   
4           0                   0             0               0   
...       ...                 ...           ...             ...   
1697        0                   0             0               0   
1698        0                   0             0               0   
1699        0                   0             0               0   
1700        0                   0             0               0   
1701        0                   0             0               0   

      Airport Service  Airport Terminal  ...  Trail  Train Station  \
0                   0                 0  ...      1              0   
1                   0                 0  ...      0              0   
2                   0                 0  ...      0              0   
3                   0                 0  ...      0              0   
4                   0                 0  ...      0              0   
...               ...               ...  ...    ...            ...   
1697                0                 0  ...      0              0   
1698                0                 0  ...      0              0   
1699                0                 0  ...      0              0   
1700                0                 0  ...      0              0   
1701                0                 0  ...      0              0   

      Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                                 0                 0                      0   
1                                 0                 0                      0   
2                                 0                 0                      0   
3                                 0                 0                      0   
4                                 0                 0                      0   
...                             ...               ...                    ...   
1697                              0                 0                      0   
1698                              0                 0                      0   
1699                              0                 0                      0   
1700                              0                 0                      0   
1701                              0                 0                      0   

      Wine Bar  Wine Shop  Wings Joint  Women's Store  Yoga Studio  
0            0     

In [17]:
toronto_grouped= toronto_onehot.groupby(['Postcode', 'Borough', 'Neighbourhood']).mean().reset_index()


toronto_grouped

Postcode           Borough  \
0       M4E      East Toronto   
1       M4K      East Toronto   
2       M4L      East Toronto   
3       M4M      East Toronto   
4       M4N   Central Toronto   
5       M4P   Central Toronto   
6       M4R   Central Toronto   
7       M4S   Central Toronto   
8       M4T   Central Toronto   
9       M4V   Central Toronto   
10      M4W  Downtown Toronto   
11      M4X  Downtown Toronto   
12      M4Y  Downtown Toronto   
13      M5A  Downtown Toronto   
14      M5B  Downtown Toronto   
15      M5C  Downtown Toronto   
16      M5E  Downtown Toronto   
17      M5G  Downtown Toronto   
18      M5H  Downtown Toronto   
19      M5J  Downtown Toronto   
20      M5K  Downtown Toronto   
21      M5L  Downtown Toronto   
22      M5N   Central Toronto   
23      M5P   Central Toronto   
24      M5R   Central Toronto   
25      M5S  Downtown Toronto   
26      M5T  Downtown Toronto   
27      M5V  Downtown Toronto   
28      M5W  Downtown Toronto   
29      M5X  Downtown Toronto   
30      M6G  Downtown Toronto   
31      M6H      West Toronto   
32      M6J      West Toronto   
33      M6K      West Toronto   
34      M6P      West Toronto   
35      M6R      West Toronto   
36      M6S      West Toronto   
37      M7A  Downtown Toronto   
38      M7Y      East Toronto   

                                        Neighbourhood  Afghan Restaurant  \
0                                         The Beaches           0.000000   
1                         The Danforth West,Riverdale           0.000000   
2                       The Beaches West,India Bazaar           0.000000   
3                                     Studio District           0.000000   
4                                       Lawrence Park           0.000000   
5                                    Davisville North           0.000000   
6                                  North Toronto West           0.000000   
7                                          Davisville           0.000000   
8                          Moore Park,Summerhill East           0.000000   
9   Deer Park,Forest Hill SE,Rathnelly,South Hill,...           0.000000   
10                                           Rosedale           0.000000   
11                         Cabbagetown,St. James Town           0.000000   
12                               Church and Wellesley           0.011628   
13                                       Harbourfront           0.000000   
14                            Ryerson,Garden District           0.000000   
15                                     St. James Town           0.000000   
16                                        Berczy Park           0.000000   
17                                 Central Bay Street           0.000000   
18                             Adelaide,King,Richmond           0.000000   
19    Harbourfront East,Toronto Islands,Union Station           0.000000   
20            Design Exchange,Toronto Dominion Centre           0.000000   
21                      Commerce Court,Victoria Hotel           0.000000   
22                                           Roselawn           0.000000   
23                 Forest Hill North,Forest Hill West           0.000000   
24                  The Annex,North Midtown,Yorkville           0.000000   
25                      Harbord,University of Toronto           0.000000   
26            Chinatown,Grange Park,Kensington Market           0.000000   
27  CN Tower,Bathurst Quay,Island airport,Harbourf...           0.000000   
28                    Stn A PO Boxes 25 The Esplanade           0.000000   
29              First Canadian Place,Underground city           0.000000   
30                                           Christie           0.000000   
31                        Dovercourt Village,Dufferin           0.000000   
32                            Little Portugal,Trinity           0.000000   
33         Brockton,Exhibition Place,Parkdale Village           0.000000   
34                    

In [18]:
num_top_venues= 10

indicators=['st','nd','rd']

area_col= ['Postcode', 'Borough','Neighbourhood']
freq_col= []

for i in np.arange(num_top_venues):
    try:
        freq_col.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        freq_col.append('{}th Most Common Venue'.format(i+1))

columns= area_col+freq_col
neighbourhood_venues_sorted= pd.DataFrame(columns=columns)
neighbourhood_venues_sorted['Postcode']= toronto_grouped['Postcode']
neighbourhood_venues_sorted['Borough']= toronto_grouped['Borough']
neighbourhood_venues_sorted['Neighbourhood']= toronto_grouped['Neighbourhood']

#print(neighbourhood_venues_sorted.head())

for i in np.arange(toronto_grouped.shape[0]):
    row_cat= toronto_grouped.iloc[i,:].iloc[3:]
    row_cat_sorted= row_cat.sort_values(ascending= False)
   # print('i=', i)
    neighbourhood_venues_sorted.iloc[i, 3:]= row_cat_sorted.index.values[0:num_top_venues]
    
neighbourhood_venues_sorted.head()

Postcode          Borough                  Neighbourhood  \
0      M4E     East Toronto                    The Beaches   
1      M4K     East Toronto    The Danforth West,Riverdale   
2      M4L     East Toronto  The Beaches West,India Bazaar   
3      M4M     East Toronto                Studio District   
4      M4N  Central Toronto                  Lawrence Park   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0                 Trail           Coffee Shop          Neighborhood   
1      Greek Restaurant           Coffee Shop    Italian Restaurant   
2                  Park        Sandwich Place        Ice Cream Shop   
3                  Café           Coffee Shop                Bakery   
4                  Park           Swim School              Bus Line   

  4th Most Common Venue   5th Most Common Venue 6th Most Common Venue  \
0                   Pub       Health Food Store      Doner Restaurant   
1        Ice Cream Shop  Furniture / Home Store    Frozen Yogurt Shop   
2     Fish & Chips Shop              Steakhouse               Brewery   
3    Italian Restaurant                 Brewery   American Restaurant   
4    Falafel Restaurant    Ethiopian Restaurant   Empanada Restaurant   

  7th Most Common Venue        8th Most Common Venue 9th Most Common Venue  \
0                 Diner               Discount Store   Distribution Center   
1                   Pub                  Pizza Place                Lounge   
2                   Pub            Food & Drink Shop  Fast Food Restaurant   
3           Yoga Studio           Seafood Restaurant        Sandwich Place   
4     Electronics Store  Eastern European Restaurant   Dumpling Restaurant   

  10th Most Common Venue  
0                Dog Run  
1           Liquor Store  
2     Italian Restaurant  
3            Cheese Shop  
4             Donut Shop

### Using KMeans to cluster the neighbourhoods

In [19]:
clusters= 5

toronto_grouped_clustering= toronto_grouped.drop(['Postcode', 'Borough', 'Neighbourhood'], 1)

kmeans= KMeans(n_clusters= clusters, random_state= 0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 0, 0, 2, 0])

In [20]:

toronto_merged = df_t.copy()

toronto_merged['Cluster Labels']= kmeans.labels_

toronto_merged= toronto_merged.join(neighbourhood_venues_sorted.drop(['Borough', 'Neighbourhood'], 1).set_index("Postcode"), on='Postcode')

toronto_merged.head()

Postcode          Borough                  Neighbourhood   Latitude  \
0      M4E     East Toronto                    The Beaches  43.676357   
1      M4K     East Toronto    The Danforth West,Riverdale  43.679557   
2      M4L     East Toronto  The Beaches West,India Bazaar  43.668999   
3      M4M     East Toronto                Studio District  43.659526   
4      M4N  Central Toronto                  Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0                 Trail           Coffee Shop   
1 -79.352188               0      Greek Restaurant           Coffee Shop   
2 -79.315572               0                  Park        Sandwich Place   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               3                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue   5th Most Common Venue  \
0          Neighborhood                   Pub       Health Food Store   
1    Italian Restaurant        Ice Cream Shop  Furniture / Home Store   
2        Ice Cream Shop     Fish & Chips Shop              Steakhouse   
3                Bakery    Italian Restaurant                 Brewery   
4              Bus Line    Falafel Restaurant    Ethiopian Restaurant   

  6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
0      Doner Restaurant                 Diner               Discount Store   
1    Frozen Yogurt Shop                   Pub                  Pizza Place   
2               Brewery                   Pub            Food & Drink Shop   
3   American Restaurant           Yoga Studio           Seafood Restaurant   
4   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0   Distribution Center                Dog Run  
1                Lounge           Liquor Store  
2  Fast Food Restaurant     Italian Restaurant  
3        Sandwich Place            Cheese Shop  
4   Dumpling Restaurant             Donut Shop

In [21]:
toronto_merged.sort_values(['Cluster Labels'], inplace=True)
toronto_merged

Postcode           Borough  \
0       M4E      East Toronto   
20      M5K  Downtown Toronto   
21      M5L  Downtown Toronto   
24      M5R   Central Toronto   
25      M5S  Downtown Toronto   
26      M5T  Downtown Toronto   
27      M5V  Downtown Toronto   
28      M5W  Downtown Toronto   
29      M5X  Downtown Toronto   
30      M6G  Downtown Toronto   
31      M6H      West Toronto   
32      M6J      West Toronto   
33      M6K      West Toronto   
34      M6P      West Toronto   
35      M6R      West Toronto   
36      M6S      West Toronto   
37      M7A  Downtown Toronto   
18      M5H  Downtown Toronto   
19      M5J  Downtown Toronto   
16      M5E  Downtown Toronto   
1       M4K      East Toronto   
2       M4L      East Toronto   
3       M4M      East Toronto   
5       M4P   Central Toronto   
6       M4R   Central Toronto   
7       M4S   Central Toronto   
17      M5G  Downtown Toronto   
9       M4V   Central Toronto   
11      M4X  Downtown Toronto   
12      M4Y  Downtown Toronto   
13      M5A  Downtown Toronto   
14      M5B  Downtown Toronto   
15      M5C  Downtown Toronto   
38      M7Y      East Toronto   
22      M5N   Central Toronto   
8       M4T   Central Toronto   
10      M4W  Downtown Toronto   
4       M4N   Central Toronto   
23      M5P   Central Toronto   

                                        Neighbourhood   Latitude  Longitude  \
0                                         The Beaches  43.676357 -79.293031   
20            Design Exchange,Toronto Dominion Centre  43.647177 -79.381576   
21                      Commerce Court,Victoria Hotel  43.648198 -79.379817   
24                  The Annex,North Midtown,Yorkville  43.672710 -79.405678   
25                      Harbord,University of Toronto  43.662696 -79.400049   
26            Chinatown,Grange Park,Kensington Market  43.653206 -79.400049   
27  CN Tower,Bathurst Quay,Island airport,Harbourf...  43.628947 -79.394420   
28                    Stn A PO Boxes 25 The Esplanade  43.646435 -79.374846   
29              First Canadian Place,Underground city  43.648429 -79.382280   
30                                           Christie  43.669542 -79.422564   
31                        Dovercourt Village,Dufferin  43.669005 -79.442259   
32                            Little Portugal,Trinity  43.647927 -79.419750   
33         Brockton,Exhibition Place,Parkdale Village  43.636847 -79.428191   
34                       High Park,The Junction South  43.661608 -79.464763   
35                              Parkdale,Roncesvalles  43.648960 -79.456325   
36                                  Runnymede,Swansea  43.651571 -79.484450   
37                                       Queen's Park  43.662301 -79.389494   
18                             Adelaide,King,Richmond  43.650571 -79.384568   
19    Harbourfront East,Toronto Islands,Union Station  43.640816 -79.381752   
16                                        Berczy Park  43.644771 -79.373306   
1                         The Danforth West,Riverdale  43.679557 -79.352188   
2                       The Beaches West,India Bazaar  43.668999 -79.315572   
3                                     Studio District  43.659526 -79.340923   
5                                    Davisville North  43.712751 -79.390197   
6                                  North Toronto West  43.715383 -79.405678   
7                                          Davisville  43.704324 -79.388790   
17                                 Central Bay Street  43.657952 -79.387383   
9   Deer Park,Forest Hill SE,Rathnelly,South Hill,...  43.686412 -79.400049   
11                         Cabbagetown,St. James Town  43.667967 -79.367675   
12                               Church and Wellesley  43.665860 -79.383160   
13                                       Harbourfront  43.654260 -79.360636   
14                            Ryerson,Garden District  43.657162 -79.378937   
15                                     St. James Town  43.651494 -79.375418

In [22]:
toronto_merged.shape

(39, 16)

In [23]:
map_clusters= folium.Map(location=[Latitude, Longitude], zoom_start=11)

x= np.arange(clusters)
ys= [i+x+(i*x)**2 for i in range(clusters)]

colors_array= cm.rainbow(np.linspace(0,1,len(ys)))
rainbow= [colors.rgb2hex(i) for i in colors_array]

markers_colors= []

for l1, l2, post, b, n, cluster in zip(toronto_merged['Latitude'],
                                         toronto_merged['Longitude'],
                                         toronto_merged['Postcode'],
                                         toronto_merged['Borough'],
                                         toronto_merged['Neighbourhood'],
                                         toronto_merged['Cluster Labels']):
    label= folium.Popup('{} ({}): {} - Cluster {}'.format(b,post,n,cluster), parse_html=True)
    folium.CircleMarker(
        [l1,l2],
        radius= 5,
        popup= label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color= rainbow[cluster-1],
        fill_opacitu= 0.7).add_to(map_clusters)
    
map_clusters

In [24]:
#cluster abs1

toronto_merged.loc[toronto_merged['Cluster Labels']== 0, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
0       East Toronto               0                 Trail   
20  Downtown Toronto               0           Coffee Shop   
21  Downtown Toronto               0           Coffee Shop   
24   Central Toronto               0                  Café   
25  Downtown Toronto               0                  Café   
26  Downtown Toronto               0                   Bar   
27  Downtown Toronto               0       Airport Service   
28  Downtown Toronto               0           Coffee Shop   
29  Downtown Toronto               0           Coffee Shop   
30  Downtown Toronto               0         Grocery Store   
31      West Toronto               0                Bakery   
32      West Toronto               0                   Bar   
33      West Toronto               0                  Café   
34      West Toronto               0    Mexican Restaurant   
35      West Toronto               0             Gift Shop   
36      West Toronto               0                  Café   
37  Downtown Toronto               0           Coffee Shop   
18  Downtown Toronto               0           Coffee Shop   
19  Downtown Toronto               0           Coffee Shop   
16  Downtown Toronto               0           Coffee Shop   
1       East Toronto               0      Greek Restaurant   
2       East Toronto               0                  Park   
3       East Toronto               0                  Café   
5    Central Toronto               0                  Park   
6    Central Toronto               0        Clothing Store   
7    Central Toronto               0        Sandwich Place   
17  Downtown Toronto               0           Coffee Shop   
9    Central Toronto               0                   Pub   
11  Downtown Toronto               0           Coffee Shop   
12  Downtown Toronto               0           Coffee Shop   
13  Downtown Toronto               0           Coffee Shop   
14  Downtown Toronto               0           Coffee Shop   
15  Downtown Toronto               0           Coffee Shop   
38      East Toronto               0            Skate Park   

    2nd Most Common Venue 3rd Most Common Venue        4th Most Common Venue  \
0             Coffee Shop          Neighborhood                          Pub   
20                   Café            Restaurant                        Hotel   
21             Restaurant                  Café                        Hotel   
24         Sandwich Place           Coffee Shop                   Donut Shop   
25                    Bar    Italian Restaurant          Japanese Restaurant   
26  Vietnamese Restaurant                  Café           Chinese Restaurant   
27         Airport Lounge      Airport Terminal                     Boutique   
28             Restaurant                  Café          Japanese Restaurant   
29                   Café            Restaurant          Japanese Restaurant   
30                   Café                  Park                  Candy Store   
31               Pharmacy               Brewery                  Music Venue   
32            Coffee Shop      Asian Restaurant                   Restaurant   
33            Coffee Shop        Breakfast Spot                          Bar   
34                    Bar                  Café              Thai Restaurant   
35          Movie Theater        Breakfast Spot  Eastern European Restaurant   
36            Coffee Shop      Sushi Restaurant                  Pizza Place   
37                   Park                   Bar           Seafood Restaurant   
18             Restaurant       Thai Restaurant                         Café   
19               Aquarium                  Café                        Hotel   
16           Cocktail Bar    Seafood Restaurant                     Beer Bar   
1             Coffee Shop    Italian Restaurant               Ice Cream Shop   
2          Sandwich Place        Ice Cream Shop            Fish & Chips Shop   
3 

In [25]:
#cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels']== 1, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
22  Central Toronto               1        Ice Cream Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
22                Garden          Dessert Shop  Ethiopian Restaurant   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
22   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
22   Dumpling Restaurant            Donut Shop       Doner Restaurant

In [26]:
#cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels']==2, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
8    Central Toronto               2                  Park   
10  Downtown Toronto               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8             Playground          Tennis Court  Colombian Restaurant   
10            Playground                 Trail      Department Store   

   5th Most Common Venue 6th Most Common Venue        7th Most Common Venue  \
8           Dessert Shop   Empanada Restaurant            Electronics Store   
10   Empanada Restaurant     Electronics Store  Eastern European Restaurant   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
8   Eastern European Restaurant   Dumpling Restaurant             Donut Shop  
10          Dumpling Restaurant            Donut Shop       Doner Restaurant

In [27]:
#cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels']== 3, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
4  Central Toronto               3                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
4           Swim School              Bus Line    Falafel Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
4  Ethiopian Restaurant   Empanada Restaurant     Electronics Store   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
4  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

In [28]:
#cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels']== 4, toronto_merged.columns[[1]+ list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
23  Central Toronto               4                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23         Jewelry Store                 Trail      Sushi Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23          Dessert Shop   Empanada Restaurant     Electronics Store   

          8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
23  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

### Observation: 
Cluster 1 is the biggest cluster and it has venues including Cafes, restaurant, supermarkets etc. whereas clusters 4 and 5 have only park and swimming pool and a park and a trail in them respectively. Cluster 2 only has a garden and Cluster 3 has only a Playground and Park